In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import Label

from ipywidgets import interact, FloatSlider

from mylib.model import thermal_explosion_model
import mylib.continuation as continuation

import warnings
warnings.filterwarnings('ignore')

output_notebook(hide_banner=True)

# Thermal explosion

In the first PC, we have considered the dynamical system modelint thermal explosion in the large activation energy and large heat of reaction limit:

$$
{\mathrm d}_\tau \widetilde\theta = e^{\widetilde\theta} -  \alpha_0\,\widetilde\theta.
$$

For the purpose of studying the branches of equilibrium points and limit points, we rather consider the following
form of the system:

$$
{\mathrm d}_t \theta = {\mathrm F}_{\mathrm k} \, e^{\theta} - \theta
$$

where the Frank-Kamemetskii parameter ${\mathrm F}_{\mathrm k} = 1/\alpha_0$ and which is obtained by a simple change of the temporal scale. The ${\mathrm F}_{\mathrm k}$ parameter is here taken as the bifurcation parameter.

In [2]:
def plot_sol():
    
    fk_lim = 1/np.exp(1)  
    
    fk = fk_lim

    tem = thermal_explosion_model(fk)
    fcn = tem.fcn
    jac = tem.jac
    
    tini = 0. 
    tend = 10.
    
    yini = (0.,)
    
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y = fig_sol.line(sol.t, sol.y[0], line_width=2)
    fig_sol.legend.location = "top_left"
    
    s_ini = 0.
    s_end = 10.
    ns = 201 
    fk_ini = 0.
    yeq_ini = (0.,)
    fkeq, yeq = continuation.pseudo_arclength_continuation(s_ini, s_end, ns, fk_ini, yeq_ini, thermal_explosion_model)

    yeq1 = np.array([0.])
    for it_newton in range(100):
        yeq1 += np.linalg.solve(jac(0, yeq1), -fcn(0, yeq1))
        if (np.linalg.norm(fcn(0, yeq1)) < 1.e-7):
            break

    yeq2 = np.array([9.5])
    for it_newton in range(100):
        yeq2 += np.linalg.solve(jac(0, yeq2), -fcn(0, yeq2))
        if (np.linalg.norm(fcn(0, yeq2)) < 1.e-7):
            break
    
    fig_eq = figure(width=475, height=300, title="Equilibrium points")
    fig_eq.line(fkeq, yeq[0])
    plt_eq = fig_eq.x(np.array([fk,fk]), np.array([yeq1[0], yeq2[0]]), line_width=2, size=10)
    
    fig_eig = figure(x_range=(0, 0.4), y_range=(-2, 7), width=475, height=300, 
                    title="Eigenvalue at each equilibrium point")
    plt_eig_2 = fig_eig.x((fk,), (yeq2[0],), line_width=2, size=10, color="red", legend="unstable point")
    plt_eig_1 = fig_eig.x((fk,), (yeq1[0],), line_width=2, size=10, color="green", legend="stable point")

    show(column(fig_sol, row(fig_eq, fig_eig)), notebook_handle=True)
    
    def update(fk):
        
        tem = thermal_explosion_model(fk)
        fcn = tem.fcn
        jac = tem.jac
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        
        yeq1 = np.array([0.])
        for it_newton in range(100):
            yeq1 += np.linalg.solve(jac(0, yeq1), -fcn(0, yeq1))
            if (np.linalg.norm(fcn(0, yeq1)) < 1.e-7):
                break

        yeq2 = np.array([9.5])
        for it_newton in range(100):
            yeq2 += np.linalg.solve(jac(0, yeq2), -fcn(0, yeq2))
            if (np.linalg.norm(fcn(0, yeq2)) < 1.e-7):
                break

        plt_y.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_eq.data_source.data = dict(x=np.array([fk,fk]), y=np.array([yeq1[0], yeq2[0]]))
        plt_eig_2.data_source.data = dict(x=(fk,), y=(yeq2[0]-1,))
        plt_eig_1.data_source.data = dict(x=(fk,), y=(yeq1[0]-1,))

        push_notebook()

    interact(update, fk=FloatSlider(min=0.01 , max=fk_lim, value=fk_lim, step=(fk_lim-0.01)/100, 
                                    continuous_update=False));

plot_sol()

interactive(children=(FloatSlider(value=0.36787944117144233, continuous_update=False, description='fk', max=0.…

In [3]:
def show_pseudo_arclength_continuation():
    
    s_ini = 0
    s_end = 10 
    ns = 101
    
    fk_ini = 0.
    theta_eq_ini = (0., )
    
    fk, theta_eq = continuation.pseudo_arclength_continuation(s_ini, s_end, ns, fk_ini, theta_eq_ini, 
                                                              thermal_explosion_model)
                
    theta_stable   = theta_eq[np.where(theta_eq < 1)]
    theta_unstable = theta_eq[np.where(theta_eq > 1)]
    fk_stable   = fk[np.where(theta_eq < 1)[1]]
    fk_unstable   = fk[np.where(theta_eq > 1)[1]]

    theta_lim = (theta_unstable[0] + theta_stable[-1])/2  
    fk_lim = (fk_unstable[0] + fk_stable[-1])/2

    fig_branch = figure(plot_height=500, plot_width=950, x_range=(0.0,0.425), y_range=(-1,10))    
    fig_branch.x(fk_stable, theta_stable, color="green", legend="stable branch")
    fig_branch.x(fk_unstable, theta_unstable, color="red", legend="unstable branch")
    fig_branch.line((0,0.425), (theta_lim, theta_lim), line_dash="dashed")
    fig_branch.line((1/np.exp(1),1/np.exp(1)), (-1, 10), line_dash="dashed")
    fig_branch.add_layout(Label(x=fk_lim+0.005, y=theta_lim+0.05, text="Limit point"))
    fig_branch.legend.location = "top_left"

    show(fig_branch)

show_pseudo_arclength_continuation()